In [27]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz
import pickle
import re
import datetime
import sympy

# Data Cleaning

### Betfair odds (scraped_dict) 

In [28]:
bookie = 'betfair'   # 'betfair', 'tipico', 'bwin'
scraped_dict = pickle.load(open(f'./scraped/dict_{bookie}.pck', 'rb'))
print(f"Length: {len(scraped_dict)}. Keys: {scraped_dict.keys()}")

# Later: Load em all
# bookie = 'betfair'   # 'betfair', 'tipico', 'bwin'
# scraped_dict_1 = pickle.load(open(f'./scraped/dict_{bookie}.pck', 'rb'))
# print(f"Length: {len(scraped_dict_1)}. Keys: {scraped_dict_1.keys()}")

# bookie = 'tipico'   # 'betfair', 'tipico', 'bwin'
# scraped_dict_2 = pickle.load(open(f'./scraped/dict_{bookie}.pck', 'rb'))
# print(f"Length: {len(scraped_dict_2)}. Keys: {scraped_dict_2.keys()}")

# bookie = 'bwin'   # 'betfair', 'tipico', 'bwin'
# scraped_dict_3 = pickle.load(open(f'./scraped/dict_{bookie}.pck', 'rb'))
# print(f"Length: {len(scraped_dict_3)}. Keys: {scraped_dict_3.keys()}")

Length: 1. Keys: dict_keys(['Germany Bundesliga'])


In [29]:
scraped_dict['Germany Bundesliga']


,,3-way,btts
Dates,Teams,,
2022-04-17,Arminia Bielefeld\nBayern Munich,12.0\n7.5\n1.14,1.68\n2.0
2022-04-16,Augsburg\nHertha Berlin,2.05\n3.3\n3.2,1.58\n2.15
2022-04-06,Augsburg\nMainz,3.2\n3.1\n2.1,1.72\n1.95
2022-04-09,Bayern Munich\nAugsburg,1.06\n10.0\n25.0,1.76\n1.9
2022-04-10,Bochum\nLeverkusen,4.0\n4.0\n1.7,1.52\n2.3
2022-04-16,Dortmund\nWolfsburg,1.46\n4.2\n6.0,1.62\n2.1
2022-04-10,Eintracht Frankfurt\nFreiburg,2.3\n3.3\n2.7,1.52\n2.3
2022-04-09,FC Koln\nMainz,2.35\n3.2\n2.7,1.58\n2.15
2022-04-16,Freiburg\nBochum,1.6\n3.8\n4.8,1.72\n1.95


In [30]:

#initialize storage (we'll use these dictionaries to match names between betfair and historical_data)
dict_home_name_matching = {}
dict_away_name_matching = {}
#fill the dictionary with a list of names of all home and away teams that will play during the week
for league in scraped_dict:
    scraped_dict[league] = scraped_dict[league].reset_index()
    scraped_dict[league][['home_team', 'away_team']] = scraped_dict[league]['Teams'].str.extract(r'(.+)\n(.+)')
    dict_home_name_matching[league] = scraped_dict[league].groupby('home_team', as_index=False).count()[['home_team']]
    dict_away_name_matching[league] = scraped_dict[league].groupby('away_team', as_index=False).count()[['away_team']]
print("done")

done


In [31]:
scraped_dict['Germany Bundesliga']

,Dates,Teams,3-way,btts,home_team,away_team
0,2022-04-17,Arminia Bielefeld\nBayern Munich,12.0\n7.5\n1.14,1.68\n2.0,Arminia Bielefeld,Bayern Munich
1,2022-04-16,Augsburg\nHertha Berlin,2.05\n3.3\n3.2,1.58\n2.15,Augsburg,Hertha Berlin
2,2022-04-06,Augsburg\nMainz,3.2\n3.1\n2.1,1.72\n1.95,Augsburg,Mainz
3,2022-04-09,Bayern Munich\nAugsburg,1.06\n10.0\n25.0,1.76\n1.9,Bayern Munich,Augsburg
4,2022-04-10,Bochum\nLeverkusen,4.0\n4.0\n1.7,1.52\n2.3,Bochum,Leverkusen
5,2022-04-16,Dortmund\nWolfsburg,1.46\n4.2\n6.0,1.62\n2.1,Dortmund,Wolfsburg
6,2022-04-10,Eintracht Frankfurt\nFreiburg,2.3\n3.3\n2.7,1.52\n2.3,Eintracht Frankfurt,Freiburg
7,2022-04-09,FC Koln\nMainz,2.35\n3.2\n2.7,1.58\n2.15,FC Koln,Mainz
8,2022-04-16,Freiburg\nBochum,1.6\n3.8\n4.8,1.72\n1.95,Freiburg,Bochum
9,2022-04-09,Greuther Furth\nMgladbach,4.2\n3.8\n1.7,1.54\n2.25,Greuther Furth,Mgladbach


### Historical Data (dict_historical_data)

In [32]:
hist_outfile_name = './scraped/dict_historic.pck'

In [33]:
# Either download data from the web (takes a while, uncomment to use)...
# dict_countries = {
#               'Germany Bundesliga':'D1', 
#               'Germany 2. Bundesliga':'D2',
#               'Italy Serie A':'I1', 
#               'Italy Serie B':'I2',
#               'Spain La Liga':'SP1', 
#               'Spain Segunda Division':'SP2',
#               'England Premier League':'E0', 
#               'England League 1':'E2', 
#               'England League 2':'E3',
#               'France Ligue 1': 'F1', 
#               'France Ligue 2':'F2'
#              }

# #  will be filled with data of the past 5 years
# base_url = "https://www.football-data.co.uk/mmz4281"
# dict_historical = {} 
# for league in dict_countries:
#     frames = []
#     for i in range(17, 22):
#         season = f"{i}{i+1}"
#         full_url = f"{base_url}/{season}/{dict_countries[league]}.csv"
#         try:
#             df = pd.read_csv(full_url)
#         except: # Italian Serie B has an inconsistent encoding!
#             df = pd.read_csv(full_url, encoding='unicode_escape')
#         df = df.assign(season=i)
#         frames.append(df)
#     df_frames = pd.concat(frames)
#     df_frames = df_frames.rename(columns={'Date':'date', 'HomeTeam':'home_team', 'AwayTeam':'away_team',
#                         'FTHG': 'home_goals', 'FTAG': 'away_goals'})
#     dict_historical[league] = df_frames

#     # save file
#     output = open(hist_outfile_name, 'wb')
#     pickle.dump(dict_historical, output)
#     output.close()


In [34]:
# ...or load from disk if downloaded before
input_file = open(hist_outfile_name,'rb')
dict_historical = pickle.load(input_file)
input_file.close()

In [35]:
# print some info
print(f"Length: {len(dict_historical)}. Keys: {dict_historical.keys()}")

Length: 11. Keys: dict_keys(['Germany Bundesliga', 'Germany 2. Bundesliga', 'Italy Serie A', 'Italy Serie B', 'Spain La Liga', 'Spain Segunda Division', 'England Premier League', 'England League 1', 'England League 2', 'France Ligue 1', 'France Ligue 2'])


In [36]:
dict_historical['Germany Bundesliga'].info()
dict_historical['Germany Bundesliga'].tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1474 entries, 0 to 249
Columns: 127 entries, Div to AvgCAHA
dtypes: float64(103), int64(17), object(7)
memory usage: 1.4+ MB


,Div,date,home_team,away_team,home_goals,away_goals,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
245,D1,02/04/2022,Hoffenheim,Bochum,1,2,A,0,1,A,...,2.52,-1.25,1.99,1.91,2.00,1.92,2.13,1.94,2.02,1.85
246,D1,02/04/2022,Leverkusen,Hertha,2,1,H,2,1,H,...,2.34,-1.25,2.00,1.90,2.00,1.93,2.01,1.96,1.95,1.91
247,D1,02/04/2022,Dortmund,RB Leipzig,1,4,A,0,2,A,...,2.42,-0.25,2.03,1.87,2.05,1.88,2.13,2.01,2.02,1.85
248,D1,03/04/2022,Augsburg,Wolfsburg,3,0,H,1,0,H,...,1.81,0.00,1.86,2.04,1.90,2.03,1.95,2.13,1.88,2.00
249,D1,03/04/2022,M'gladbach,Mainz,1,1,D,1,0,H,...,2.07,-0.25,2.09,1.81,2.10,1.84,2.12,1.93,2.06,1.81


### Matching team names. Replacing "Historical Data" team names in  betfair dataframes

In [37]:
# before matching
print(f"{len(scraped_dict['Germany Bundesliga']['home_team'].unique())}")
scraped_dict['Germany Bundesliga']['home_team'].unique().tolist()

18


['Arminia Bielefeld',
 'Augsburg',
 'Bayern Munich',
 'Bochum',
 'Dortmund',
 'Eintracht Frankfurt',
 'FC Koln',
 'Freiburg',
 'Greuther Furth',
 'Hertha Berlin',
 'Hoffenheim',
 'Leverkusen',
 'Mainz',
 'Mgladbach',
 'RB Leipzig',
 'Stuttgart',
 'Union Berlin',
 'Wolfsburg']

In [38]:
for league in dict_historical:
    #print(f"League: {league}")

    #picking unique team names inside the historical_data team names
    all_teams = dict_historical[league]['home_team'].unique().tolist()
    #print(f"{all_teams}")

    if league not in dict_home_name_matching:
        continue

    #matching betfair names (scraped_dict -> dict_home_name_matching, dict_away_name_matching) with historical data (dict_historical_data -> all_teams)
    match = lambda x:process.extractOne(x, all_teams, scorer=fuzz.token_set_ratio, score_cutoff=10)
    match_no_nan = lambda x:match(x) if match(x) is not None else x

    dict_home_name_matching[league][['teams_matched', 'score']] = dict_home_name_matching[league]['home_team'].apply(match_no_nan).apply(pd.Series)
    dict_away_name_matching[league][['teams_matched', 'score']] = dict_away_name_matching[league]['away_team'].apply(match_no_nan).apply(pd.Series)
    
    # TODO: Set all available market columns below!!! 

    #Replacing "Historical Data" team names (teams_matched) in  betfair dataframes
    home_teams = pd.merge(scraped_dict[league], dict_home_name_matching[league], on='home_team',
                          how='left')[['Dates', '3-way', 'btts', 'teams_matched']].rename(columns={'teams_matched':'home_team'})
    away_teams = pd.merge(scraped_dict[league], dict_away_name_matching[league], on='away_team',
                         how='left')[['teams_matched']].rename(columns={'teams_matched':'away_team'})
    
    #updating values
    scraped_dict.update({league:pd.concat([home_teams, away_teams], axis=1)})


In [39]:
# check results
dict_leagues = {
               'Germany Bundesliga'     : ('german football','German Bundesliga'),
               'Germany 2. Bundesliga'  : ('german football', 'German Bundesliga 2'),
               'Italy Serie A'          : ('italian football', 'Italian Serie A'),
               'Italy Serie B'          : ('italian football', 'Italian Serie B'),
               'Spain La Liga'          : ('spanish football', 'Spanish La Liga'),
               'Spain Segunda Division' : ('spanish football', 'Spanish Segunda Division'),
               'England Premier League' : ('english football', 'English Premier League'), 
               'England League 1'       : ('english football', 'English League 1'),
               'England League 2'       : ('english football', 'English League 2'),
               'France Ligue 1'         : ('french football', 'French Ligue 1'),
               'France Ligue 2'         : ('french football','French Ligue 2'),
               } 

dict_home_name_matching['Germany Bundesliga']

,home_team,teams_matched,score
0,Arminia Bielefeld,Bielefeld,100
1,Augsburg,Augsburg,100
2,Bayern Munich,Bayern Munich,100
3,Bochum,Bochum,100
4,Dortmund,Dortmund,100
5,Eintracht Frankfurt,Ein Frankfurt,82
6,FC Koln,FC Koln,100
7,Freiburg,Freiburg,100
8,Greuther Furth,Greuther Furth,100
9,Hertha Berlin,Hertha,100


In [ ]:
# after matching
print(f"{len(scraped_dict['Germany Bundesliga']['home_team'].unique())}")
scraped_dict['Germany Bundesliga']['home_team'].unique().tolist()

In [ ]:
# store
outfile_name = f'./scraped/dict_{bookie}_renamed.pck'
output = open(outfile_name, 'wb')
pickle.dump(scraped_dict, output)
output.close()
print(f"Done. Stored to {outfile_name}")

# Meine Surebet Experimente

In [ ]:
#1.#transforming data bookie 1,2 and 3
df_betfair = pickle.load(open('./scraped/dict_betfair_renamed.pck','rb'))['Germany Bundesliga']
df_betfair = df_betfair.set_index(["Dates", "home_team", "away_team"])
df_betfair = df_betfair.replace(r'', '0\n0', regex=True)
df_betfair = df_betfair.replace(r'^\d+\.\d+$', '0\n0', regex=True)
df_betfair = df_betfair.sort_index()

df_tipico = pickle.load(open('./scraped/dict_tipico_renamed.pck','rb'))['Germany Bundesliga']
df_tipico = df_tipico.set_index(["Dates", "home_team", "away_team"])
df_tipico = df_tipico.replace(r'', '0\n0', regex=True)#odds with no values
df_tipico = df_tipico.replace(r'^\d+\.\d+$', '0\n0', regex=True)#odds with only one element
df_tipico = df_tipico.sort_index()

df_bwin = pickle.load(open('./scraped/dict_bwin_renamed.pck','rb'))['Germany Bundesliga']
df_bwin = df_bwin.set_index(["Dates", "home_team", "away_team"])
df_bwin = df_bwin.replace(r'', '0\n0', regex=True)
df_bwin = df_bwin.replace(r'^\d+\.\d+$', '0\n0', regex=True)
df_bwin = df_bwin.sort_index()


#df_betfair
#df_tipico
#df_bwin

In [ ]:
def score(val1, val2, val3):
  val = 1.0/(1.0/val1 + 1.0/val2 + 1.0/val3)
  return f"{val:.2f}"

best_odds = {}

for df in [df_betfair, df_tipico, df_bwin]:
  for index,row in df.iterrows():
    date, home_team, away_team = index
    odd_1, odd_X, odd_2 = [float(x) for x in row["3-way"].split('\n')]
    if not index in best_odds.keys():
      #print(f"Found new game: {date}: {home_team} vs {away_team} -> Odds: {odd_1} / {odd_X} / {odd_2}. Score: { score(odd_1,odd_X,odd_2) }")
      best_odds[index] = (odd_1, odd_X, odd_2)
    else:
      #print(f"Game already exists: {date}: {home_team} vs {away_team}")      
      old_odds = best_odds[index]
      old_score = score(old_odds[0], old_odds[1], old_odds[2])
      improved = False

      if best_odds[index][0] < odd_1:
        best_odds[index] = (odd_1, best_odds[index][1], best_odds[index][2])
        improved = True
      if best_odds[index][1] < odd_X:
        best_odds[index] = (best_odds[index][0], odd_X, best_odds[index][2])
        improved = True
      if best_odds[index][2] < odd_2:
        best_odds[index] = (best_odds[index][0], best_odds[index][1], odd_2)
        improved = True

      if improved:
        new_odds = best_odds[index]
        new_score = score(new_odds[0], new_odds[1], new_odds[2])
        #print(f"  Improved Odds: {old_odds} --> {new_odds}. Improved score: {old_score} --> {new_score}")

for index, odds in best_odds.items():
  date, home_team, away_team = index
  odd_1, odd_X, odd_2 = odds
  curr_score = float(score(odds[0], odds[1], odds[2]))
  if curr_score > 1.0:
    print(f"Found surebet: {date}: {home_team} vs {away_team} -> Odds: {odd_1} / {odd_X} / {odd_2}. Score: { curr_score }")
  #else:
  #  print(f"NO surebet: {date}: {home_team} vs {away_team} -> Odds: {odd_1} / {odd_X} / {odd_2}. Score: { curr_score }")

In [ ]:
#2.String matching
teams_1 = df_tipico['Teams'].tolist()
teams_2 = df_bwin['Teams'].tolist()
teams_3 = df_betfair['Teams'].tolist()

#team names and scores matched
df_tipico[['Teams_matched_bwin', 'Score_bwin']] = df_tipico['Teams'].apply(lambda x:process.extractOne(x, teams_2, scorer=fuzz.token_set_ratio)).apply(pd.Series)
df_tipico[['Teams_matched_betfair', 'Score_betfair']] = df_tipico['Teams'].apply(lambda x:process.extractOne(x, teams_3, scorer=fuzz.token_set_ratio)).apply(pd.Series)
df_bwin[['Teams_matched_betfair', 'Score_betfair']] = df_bwin['Teams'].apply(lambda x:process.extractOne(x, teams_3, scorer=fuzz.token_set_ratio)).apply(pd.Series)

df_surebet_tipico_bwin = pd.merge(df_tipico, df_bwin, left_on='Teams_matched_bwin', right_on='Teams')
df_surebet_tipico_bwin = df_surebet_tipico_bwin[df_surebet_tipico_bwin['Score_bwin']>60]
df_surebet_tipico_bwin = df_surebet_tipico_bwin[['Teams_x', 'btts_x', 'Teams_y', 'btts_y']]

df_surebet_tipico_betfair = pd.merge(df_tipico, df_betfair, left_on='Teams_matched_betfair', right_on='Teams')
df_surebet_tipico_betfair = df_surebet_tipico_betfair[df_surebet_tipico_betfair['Score_betfair']>60]
df_surebet_tipico_betfair = df_surebet_tipico_betfair[['Teams_x', 'btts_x', 'Teams_y', 'btts_y']]

df_surebet_bwin_betfair = pd.merge(df_bwin, df_betfair, left_on='Teams_matched_betfair', right_on='Teams')
df_surebet_bwin_betfair = df_surebet_bwin_betfair[df_surebet_bwin_betfair['Score_betfair']>60]
df_surebet_bwin_betfair = df_surebet_bwin_betfair[['Teams_x', 'btts_x', 'Teams_y', 'btts_y']]




In [ ]:
#3,Finding Surebets
#Formula to find surebets
def find_surebet(frame):
    frame[['btts_x_1', 'btts_x_2']] = frame['btts_x'].apply(lambda x: x.split('\n')).apply(pd.Series).astype(float)
    frame[['btts_y_1', 'btts_y_2']] = frame['btts_y'].apply(lambda x: x.split('\n')).apply(pd.Series).astype(float)
    frame['sure_btts1'] = (1 / frame['btts_x_1']) + (1 / frame['btts_y_2'])
    frame['sure_btts2'] = (1 / frame['btts_x_2']) + (1 / frame['btts_y_1'])
    frame = frame[['Teams_x', 'btts_x', 'Teams_y', 'btts_y', 'sure_btts1', 'sure_btts2']]
    frame = frame[(frame['sure_btts1'] < 1.2) | (frame['sure_btts2'] < 1.2)]
    frame.reset_index(drop=True, inplace=True)
    return frame

#applying formula
df_surebet_tipico_bwin = find_surebet(df_surebet_tipico_bwin)
df_surebet_tipico_betfair = find_surebet(df_surebet_tipico_betfair)
df_surebet_bwin_betfair = find_surebet(df_surebet_bwin_betfair)

#creating dictionary
dict_surebet = {'Tipico-Bwin':df_surebet_tipico_bwin, 'Tipico-Betfair':df_surebet_tipico_betfair, 'Bwin-Betfair':df_surebet_bwin_betfair}


In [ ]:
dict_surebet

In [ ]:
# 4. Calculate the stakes for betting
def beat_bookies(odds1, odds2, total_stake):
    x, y = symbols('x y')
    eq1 = Eq(x + y - total_stake, 0) # total_stake = x + y
    eq2 = Eq((odds2*y) - odds1*x, 0) # odds1*x = odds2*y
    stakes = solve((eq1,eq2), (x, y))
    total_investment = stakes[x] + stakes[y]
    profit1 = odds1*stakes[x] - total_stake
    profit2 = odds2*stakes[y] - total_stake
    benefit1 = f'{profit1 / total_investment * 100:.2f}%'
    benefit2 = f'{profit2 / total_investment * 100:.2f}%'
    dict_gabmling = {'Odds1':odds1, 'Odds2':odds2, 'Stake1':f'${stakes[x]:.0f}', 'Stake2':f'${stakes[y]:.0f}', 'Profit1':f'${profit1:.2f}', 'Profit2':f'${profit2:.2f}',
                    'Benefit1': benefit1, 'Benefit2': benefit2}
    return dict_gabmling

total_stake = 100 #set your total stake
for frame in dict_surebet:
    if len(dict_surebet[frame])>=1:
        print('------------------SUREBETS Found! '+ frame +' (check team names)--------------------------------------------------')
        print(dict_surebet[frame])
        print('------------------Stakes-------------------------')
        for i, value in enumerate(dict_surebet[frame]['sure_btts1']):
            if value<1:
                odds1 = float(dict_surebet[frame].at[i, 'btts_x'].split('\n')[0])
                odds2 = float(dict_surebet[frame].at[i, 'btts_y'].split('\n')[1])
                teams = dict_surebet[frame].at[i, 'Teams_x'].split('\n')
                dict_1 = beat_bookies(odds1, odds2, total_stake)
                print(str(i)+' '+'-'.join(teams)+ ' ----> '+ ' '.join('{}:{}'.format(x, y) for x,y in dict_1.items()))
        for i, value in enumerate(dict_surebet[frame]['sure_btts2']):
            if value<1:
                odds1 = float(dict_surebet[frame].at[i, 'btts_x'].split('\n')[1])
                odds2 = float(dict_surebet[frame].at[i, 'btts_y'].split('\n')[0])
                teams = dict_surebet[frame].at[i, 'Teams_x'].split('\n')
                dict_2 = beat_bookies(odds1, odds2, total_stake)
                print(str(i) + ' ' + '-'.join(teams) + ' ----> ' + ' '.join('{}:{}'.format(x, y) for x, y in dict_2.items()))


Ab hier sind Value Bet und Stats Dinge, hab ich mir bis jetzt nicht angesehen...

# Creating Profiles to calculate Real odds and Stats

In [ ]:
#df_historical: 5 years for h2h analysis (it doesn't have 'HST', 'AST', 'HC', 'AC') Only recent seasons have all columns available
df_historical = pd.concat(dict_historical, ignore_index=True)
df_historical = df_historical[['date', 'home_team', 'away_team', 'home_goals', 'away_goals', 'season']] 


In [ ]:
#df_profile: 2 years (current + last season) for real odds and stats calculation (it has 'HST', 'AST', 'HC', 'AC')
#You can add or omit seasons    
seasons = [19,20]

df_profile = pd.concat(dict_historical, ignore_index=True)
df_profile = df_profile[df_profile['season'].isin(seasons)]
df_profile = df_profile[['date', 'home_team', 'away_team', 'home_goals', 'away_goals',
                               'HST', 'AST', 'HC', 'AC', 'season']]
df_profile = df_profile.rename(columns={'HST':'home_shots_target', 'AST':'away_shots_target',
                                              'HC':'home_corners', 'AC':'away_corners'})

df_profile

# Calculating Stats

In [ ]:
#My method
# df_stats = pd.concat([
#     df_profile.groupby('home_team').mean().round(2)[['home_goals', 'home_shots_target', 'home_corners']],
#     df_profile.groupby('away_team').mean().round(2)[['away_goals', 'away_shots_target', 'away_corners']]],
#     axis=1)

#Betpractice method for average goals
df_home = df_profile.groupby('home_team').mean().round(2)
df_home['home_goals_betpractice'] = df_home['home_goals'] + df_home['away_goals']
df_away = df_profile.groupby('away_team').mean().round(2)
df_away['away_goals_betpractice'] = df_away['home_goals'] + df_away['away_goals']

df_stats = pd.concat([
    df_home[['home_goals_betpractice', 'home_shots_target', 'home_corners']],
    df_away[['away_goals_betpractice', 'away_shots_target', 'away_corners']],
], axis=1)

df_stats.rename(columns={'home_goals_betpractice':'home_goals', 'away_goals_betpractice':'away_goals'},
               inplace=True)
df_stats

# Creating Market Filters

### Filter 1 (Date) and Filter 2 (Stats/Presets) (run from here)

###### Note: The following error "You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat" happens when df_betfair doesn't have data available for the date you're selecting (the data scrape only have games for the next 4 days or so)

In [ ]:
#inputs
match_date = '2022-04-02'
dict_filter = {'goals':2.5, 'shots_target':9, 'corners':9}
#-----------------------------------
#concatenate all leagues in scraped_dict
df_betfair = pd.concat(scraped_dict, ignore_index=True)
df_betfair = df_betfair[df_betfair['Dates']==match_date]
#split home and away teams
home = df_betfair['home_team'].to_list()
away = df_betfair['away_team'].to_list()

goals = []
shots_target = []
corners = []
promoted_team = [] #no stats available (promoted teams that don't have stats available in the recent 2 years)
#calculating and storing stats (average goals, total shots on target and total corners)
for home_team, away_team in zip(home, away):
    try:
        goals.append(round((df_stats.loc[home_team, 'home_goals'] + df_stats.loc[away_team, 'away_goals'])/2, 2))#betpractice method
        shots_target.append(df_stats.loc[home_team, 'home_shots_target'] + df_stats.loc[away_team, 'away_shots_target'])
        corners.append(df_stats.loc[home_team, 'home_corners'] + df_stats.loc[away_team, 'away_corners'])       
    except:
        print('Possible promoted teams: '+home_team+' - '+away_team) #no stats data
        goals.append(0)
        shots_target.append(0)
        corners.append(0)

#filtering games out            
df_filters = pd.DataFrame.from_dict({'home_team':home, 'away_team':away, 'goals':goals,
                                     'shorts_target':shots_target, 'corners':corners})
df_filters = df_filters[(df_filters['goals']>=dict_filter['goals']) &
                        (df_filters['shorts_target']>=dict_filter['shots_target']) &
                        (df_filters['corners']>=dict_filter['corners'])]
df_filters = pd.merge(df_betfair, df_filters, on=['home_team', 'away_team'])

### Real Odds

In [ ]:
def calculate_real_odds(home_team, away_team, df_profile, market):
    """Calculates the reals odds in a match with data based on selected seasons (df_profile)"""
    df_profile = df_profile.assign(total_goals=df_profile['home_goals'] + df_profile['away_goals'])
    df_profile['Over/Under'] = np.where(df_profile['total_goals']>2, 'Over 2.5', 'Under 2.5')
    df_profile['BTTS'] = np.where((df_profile['home_goals']>0) & (df_profile['away_goals']>0), True, False)
    dict_markets = {'BTTS':[True, False], 'Over/Under':['Over 2.5', 'Under 2.5']}
    df_real_odds = df_profile[(df_profile['home_team']==home_team)|(df_profile['away_team']==away_team)].groupby(market).count()[['total_goals']]
    option1 = df_real_odds.loc[dict_markets[market][0], 'total_goals']#over2.5
    option2 = df_real_odds.loc[dict_markets[market][1], 'total_goals']#under2.5
    percentage_odds_over = option1 / (option1 + option2)
    real_odds = round(1/percentage_odds_over, 2)
    return real_odds

### Filter 3: Real Odds: BTTS, Over 2.5 <= 2

In [ ]:
#inputs
dict_filter_real_odds = {'real_odds_over':2, 'real_odds_btts':2} #at least 50% of chances
#-------
#simplifying formula
calculate_over = lambda x, y : calculate_real_odds(x, y, df_profile, 'Over/Under')
calculate_btts = lambda x, y : calculate_real_odds(x, y, df_profile, 'BTTS')
#aplying formula to calculate reald odds
df_filters['real_odds_over'] = df_filters.apply(lambda x:calculate_over(x['home_team'], x['away_team']), axis=1)
df_filters['real_odds_btts'] = df_filters.apply(lambda x:calculate_btts(x['home_team'], x['away_team']), axis=1)
#filtering teams out based on reald odds
df_filters = df_filters[(df_filters['real_odds_over']<=dict_filter_real_odds['real_odds_over']) &
                        (df_filters['real_odds_btts']<=dict_filter_real_odds['real_odds_btts'])]

# Finding Value Bets

In [ ]:
#picking columns
df_system = df_filters[['home_team', 'away_team', 'goals', 'shorts_target', 'corners',
                      'real_odds_over', 'over2.5', 'real_odds_btts', 'btts']]
#renaming columns
df_system.rename(columns={'shorts_target':'SoT', 'corners':'C', 'goals':'G', 'over2.5':'OVER',
                          'btts':'BTTS', 'real_odds_over':'RO_OVER', 'real_odds_btts':'RO_BTTS'}, inplace=True)
#extracting over2.5 and btts=True values
df_system = df_system.assign(OVER = df_system['OVER'].str.extract(r'(.+)\n.+').astype(float))
df_system = df_system.assign(BTTS = df_system['BTTS'].str.extract(r'(.+)\n.+').astype(float))
#calculating value of bets
df_system['V_OVER'] = df_system['OVER'] - df_system['RO_OVER']
df_system['V_BTTS'] = df_system['BTTS'] - df_system['RO_BTTS']
#sorting matches by value
df_system.sort_values('V_OVER', ascending=False)

# Match Analysis

### Hand to Hand (5 years)

In [ ]:
#introduce index of table above
index = 5 #example inter - lazio (2021-02-14)
#-------
home_h2h = df_system.loc[index, 'home_team']
away_h2h = df_system.loc[index, 'away_team']
teams_h2h = [home_h2h, away_h2h]

df_h2h = df_historical[(df_historical['home_team'].isin(teams_h2h)) & 
           (df_historical['away_team'].isin(teams_h2h))]
df_h2h[['date',	'home_team', 'away_team', 'home_goals', 'away_goals']]

## Current season analysis

In [ ]:
current_season = max(df_profile['season'].to_list())
df_current_season = df_profile[df_profile['season']==current_season]
df_current_season

### Past 5 games home and away in current season (h2h)

In [ ]:
#last 5
pd.concat([df_current_season[df_current_season['home_team']==home_h2h][-5:], 
           df_current_season[df_current_season['away_team']==home_h2h][-5:]])

pd.concat([df_current_season[df_current_season['home_team']==away_h2h][-5:], 
           df_current_season[df_current_season['away_team']==away_h2h][-5:]])

### Goal average in current season (h2h)

In [ ]:
def calculate_current_season_average_goals(team):
    #goals scored when playing home
    a = df_current_season[df_current_season['home_team']==team]['home_goals'].mean().round(2)
    #goals conceded when playing home
    b = df_current_season[df_current_season['away_team']==team]['away_goals'].mean().round(2)

    #goals scored when playing away
    c = df_current_season[df_current_season['home_team']==team]['away_goals'].mean().round(2)
    #goals conceded when playing away
    d = df_current_season[df_current_season['away_team']==team]['home_goals'].mean().round(2)
    return pd.DataFrame({'home':[a, c], 'away':[b, d]}, index=['Scored', 'Conceded'])

calculate_current_season_average_goals(away_h2h) #home_h2h(Inter), away_h2h(Lazio)